## Batch attributes from existing tables using Signals

This notebook creates a new view using the SDK that will be computed using stream processing.

### Flow of data

```mermaid
flowchart LR
    wh[(Warehouse)]
    cron[/Materialization CRON job/]
    signals(Signals)

    wh --> cron
    cron --> signals
```

---

In [ ]:
from snowplow_signals import Signals
from dotenv import load_dotenv
import os

load_dotenv()

sp_signals = Signals(
    api_url=os.environ["SNOWPLOW_API_URL"],
    api_key=os.environ["SNOWPLOW_API_KEY"],
    api_key_id=os.environ["SNOWPLOW_API_KEY_ID"],
    org_id=os.environ["SNOWPLOW_ORG_ID"],
)

### Define a new data source

Creates a data source with the source Snowflake table configuration.

In [4]:
from snowplow_signals import BatchSource

data_source = BatchSource(
    name="ecommerce_transaction_interactions_source",
    database="SNOWPLOW_DEV1",
    schema="SIGNALS",
    table="SNOWPLOW_ECOMMERCE_TRANSACTION_INTERACTIONS_FEATURES",
    timestamp_field="UPDATED_AT",
)

### Create a view with the table fields

Views define the attributes inside the data sources and the entity types they relate to.

In [5]:
from snowplow_signals import View, user_entity, Field

view = View(
    name="ecommerce_transaction_interactions_attributes",
    version=1,
    entity=user_entity,
    fields=[
        Field(
            name="TOTAL_TRANSACTIONS",
            type="int32",
        ),
        Field(
            name="TOTAL_REVENUE",
            type="int32",
        ),
        Field(
            name="AVG_TRANSACTION_REVENUE",
            type="int32",
        ),
    ],
    batch_source=data_source,
)

### Applying the data source and view to Signals

The following block pushes the data source and view definition to the Signals API and makes it available for a background CRON job that incrementally materializes the data from the warehouse table to the online attribute store.

In [6]:
from snowplow_signals import Signals

applied = sp_signals.apply([view])
print(f"{len(applied)} objects applied")

1 objects applied


### Retrieving data

One can fetch the latest attribute values for a particular user from the view as follow.

In [8]:
response = sp_signals.get_online_attributes(
    view,
    identifiers=[
        "9999999999999999999999999",
    ],
)

response.to_dataframe()

,domain_userid,TOTAL_REVENUE,TOTAL_TRANSACTIONS,AVG_TRANSACTION_REVENUE
0,9999999999999999999999999,None,None,None
